In [1]:
# 设置文件编码格式为UTF-8，支持中文字符显示

import pandas as pd  # 数据处理库，用于读取和处理CSV文件
import jieba  # 中文分词库，用于将中文文本切分为词语
from sklearn.feature_extraction.text import TfidfVectorizer  # 文本特征提取工具，将文本转换为TF-IDF向量
from sklearn.naive_bayes import MultinomialNB  # 多项式朴素贝叶斯分类器，适合文本分类任务
from sklearn.model_selection import train_test_split  # 数据集划分工具，用于分割训练/验证/测试集
from sklearn.metrics import classification_report, confusion_matrix, f1_score  # 模型评估工具
from scipy.sparse import vstack  # 处理稀疏矩阵的垂直合并，用于合并训练集和验证集

# 定义分层划分数据集的函数
def stratified_split(data, label_col, ratios=(0.8, 0.1, 0.1)):
    """分层抽样划分数据集为训练集、验证集、测试集，保持类别比例一致"""
    # 第一步：划分测试集（占总数据的最后10%）
    X_temp, X_test, y_temp, y_test = train_test_split(
        data["text"],          # 文本数据列（特征）
        data[label_col],       # 标签列（目标变量）
        test_size=ratios[2],   # 测试集比例（第三个元素，0.1）
        stratify=data[label_col],  # 按标签分布分层抽样，确保各子集类别比例一致
        random_state=42        # 随机种子，保证结果可复现
    )

    # 第二步：将剩余数据（X_temp和y_temp）划分为训练集和验证集
    X_train, X_valid, y_train, y_valid = train_test_split(
        X_temp,               # 剩余文本数据（特征）
        y_temp,               # 剩余标签数据（目标变量）
        test_size=ratios[1]/(1 - ratios[2]),  # 计算验证集在剩余数据中的比例
        # 总比例为0.8训练，0.1验证，0.1测试 → 剩余90%中验证占1/9≈0.111
        stratify=y_temp,      # 按标签分布分层抽样
        random_state=42       # 随机种子
    )
    return X_train, X_valid, X_test, y_train, y_valid, y_test  # 返回所有划分后的数据集

# 加载数据集
data = pd.read_csv("filtered_cnews.train.csv", names=["label", "text"], header=0)
# 注意：这里CSV文件第一行是列名，设置header=0会跳过第一行并使用names参数指定的列名


# 检查原始数据分布
print("原始数据分布：")
print(data["label"].value_counts())  # 统计每个类别的样本数量，确保数据平衡性

# 自定义中文分词函数
def chinese_tokenizer(text):
    return list(jieba.cut(text))  # 使用jieba精确模式切分中文文本为词语列表

# 执行数据集划分（训练80%，验证10%，测试10%）
X_train, X_valid, X_test, y_train, y_valid, y_test = stratified_split(
    data, "label", ratios=(0.8, 0.1, 0.1)  # 划分比例：训练0.8，验证0.1，测试0.1
)

# 验证划分后的数据分布
print("\n训练集分布：")
print(y_train.value_counts())  # 检查训练集各分类样本数量
print("\n验证集分布：")
print(y_valid.value_counts())  # 检查验证集各分类样本数量
print("\n测试集分布：")
print(y_test.value_counts())   # 检查测试集各分类样本数量

# 特征工程：TF-IDF向量化
tfidf = TfidfVectorizer(
    tokenizer=chinese_tokenizer,  # 使用自定义的中文分词器
    max_features=5000,           # 最多保留5000个特征词（按重要性排序）
    ngram_range=(1, 2),          # 使用单字（unigram）和双字（bigram）组合特征
    min_df=5,                    # 忽略在少于5个文档中出现的词（过滤低频词）
    max_df=0.9                   # 忽略在超过90%文档中出现的高频词（如停用词）
)
X_train_tfidf = tfidf.fit_transform(X_train)  # 在训练集上训练并转换为TF-IDF向量
X_valid_tfidf = tfidf.transform(X_valid)      # 验证集仅转换（使用训练集的词典）
X_test_tfidf = tfidf.transform(X_test)        # 测试集仅转换

# 超参数调优（使用验证集选择最佳alpha值）
alphas = [0.1, 0.5, 1.0, 1.5, 2.0]  # 拉普拉斯平滑系数候选值
best_score = -1                      # 初始化最佳准确率
best_alpha = alphas[0]               # 初始化最佳alpha值

print("\n===== 验证集调参过程 =====")
for alpha in alphas:
    model = MultinomialNB(alpha=alpha)  # 创建朴素贝叶斯模型实例
    model.fit(X_train_tfidf, y_train)   # 在训练集上训练模型
    valid_score = model.score(X_valid_tfidf, y_valid)  # 计算验证集准确率
    print(f"alpha={alpha:.1f} 验证集准确率: {valid_score:.4f}")

    # 记录最佳参数
    if valid_score > best_score:
        best_score = valid_score
        best_alpha = alpha

print(f"\n最佳参数：alpha={best_alpha} (验证集准确率：{best_score:.4f})")

# 训练最终模型（合并训练集+验证集）
print("\n===== 最终模型训练 =====")
final_model = MultinomialNB(alpha=best_alpha)  # 使用最佳alpha值
# 合并训练集和验证集的数据
X_final = vstack([X_train_tfidf, X_valid_tfidf])  # 垂直合并稀疏矩阵（特征）
y_final = pd.concat([y_train, y_valid])           # 合并标签Series对象
final_model.fit(X_final, y_final)                 # 使用全部可用数据训练最终模型

# 测试集评估
print("\n===== 测试集评估 =====")
y_pred = final_model.predict(X_test_tfidf)  # 预测测试集标签

# 输出详细分类报告
print("\n分类报告：")
print(classification_report(y_test, y_pred, digits=4))
# 包含precision、recall、f1-score、support等指标，按类别显示

# 计算不同平均方式的F1值
macro_f1 = f1_score(y_test, y_pred, average="macro")  # 宏平均：各分类F1的均值
micro_f1 = f1_score(y_test, y_pred, average="micro")  # 微平均：整体正确率
print(f"\n宏平均F1: {macro_f1:.4f}")
print(f"微平均F1: {micro_f1:.4f}")

# 输出混淆矩阵（以DataFrame格式展示）
cm = confusion_matrix(y_test, y_pred)
print("\n混淆矩阵：")
print(pd.DataFrame(cm,
                   index=final_model.classes_,  # 真实类别（行）
                   columns=final_model.classes_))  # 预测类别（列）
# 混淆矩阵[i][j]表示：真实类别为i，预测为j的样本数量


C:\Users\86188\numpy_1x_env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86188\AppData\Local\Temp\jieba.cache


原始数据分布：
label
体育    5000
家居    5000
房产    5000
教育    5000
科技    5000
财经    5000
Name: count, dtype: int64

训练集分布：
label
家居    4000
体育    4000
财经    4000
教育    4000
科技    4000
房产    4000
Name: count, dtype: int64

验证集分布：
label
体育    500
家居    500
教育    500
房产    500
财经    500
科技    500
Name: count, dtype: int64

测试集分布：
label
财经    500
教育    500
体育    500
科技    500
房产    500
家居    500
Name: count, dtype: int64


Loading model cost 0.368 seconds.
Prefix dict has been built successfully.



===== 验证集调参过程 =====
alpha=0.1 验证集准确率: 0.9410
alpha=0.5 验证集准确率: 0.9403
alpha=1.0 验证集准确率: 0.9403
alpha=1.5 验证集准确率: 0.9393
alpha=2.0 验证集准确率: 0.9397

最佳参数：alpha=0.1 (验证集准确率：0.9410)

===== 最终模型训练 =====

===== 测试集评估 =====

分类报告：
              precision    recall  f1-score   support

          体育     1.0000    0.9920    0.9960       500
          家居     0.9708    0.9300    0.9499       500
          房产     0.9312    0.9480    0.9395       500
          教育     0.9435    0.9360    0.9398       500
          科技     0.9065    0.9700    0.9372       500
          财经     0.9464    0.9180    0.9320       500

    accuracy                         0.9490      3000
   macro avg     0.9497    0.9490    0.9491      3000
weighted avg     0.9497    0.9490    0.9491      3000


宏平均F1: 0.9491
微平均F1: 0.9490

混淆矩阵：
     体育   家居   房产   教育   科技   财经
体育  496    0    0    2    2    0
家居    0  465   22    5    6    2
房产    0    2  474    4    0   20
教育    0    5    2  468   22    3
科技    0    4    2    8  485    1